In [ ]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import Sequence

# Define constants
FRAMES_DIR = r"C:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub_Monisick_ML\CapstoneGithub\images\frames"
OVERHEAD_DIR = r"C:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub_Monisick_ML\CapstoneGithub\images\overhead"
BATCH_SIZE = 16
IMAGE_SIZE = (224, 224)  # Resize images to (224, 224) for the model input
EPOCHS = 50  # Number of epochs for training

# Load dish metadata without header row and manually assign column names
column_names = [
    'dish_id', 'total_calories', 'total_mass', 'total_fat', 'total_carb', 'total_protein'
]

# Load the CSV without headers and assign the column names
dish_data = pd.read_csv(r"C:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub_Monisick_ML\CapstoneGithub\metadata\dish_cleaned.csv", header=None, names=column_names)

# Step 1: Filter dishes with 3 or more images in the frames_sampled30 folder
def get_valid_dishes(frames_base_dir):
    dish_dirs = os.listdir(frames_base_dir)
    valid_dishes = []

    for dish_dir in dish_dirs:
        dish_path = os.path.join(frames_base_dir, dish_dir, "frames_sampled30")
        if os.path.isdir(dish_path):
            image_files = [f for f in os.listdir(dish_path) if f.endswith('.jpeg')]
            if len(image_files) >= 3:
                valid_dishes.append((dish_dir, image_files))
    
    return valid_dishes

valid_dishes = get_valid_dishes(FRAMES_DIR)

# Step 2: Split dishes into train, validation, and test sets (1 image per set)
random.shuffle(valid_dishes)

train_dishes = []
val_dishes = []
test_dishes = []

for dish_dir, images in valid_dishes:
    random.shuffle(images)
    train_dishes.append((dish_dir, images[0]))
    val_dishes.append((dish_dir, images[1]))
    test_dishes.append((dish_dir, images[2]))

class MultiInputGenerator(Sequence):
    def __init__(self, dish_data, batch_size, dish_ids):
        self.dish_data = dish_data
        self.batch_size = batch_size
        self.dish_ids = dish_ids

    def __len__(self):
        return int(np.ceil(len(self.dish_ids) / self.batch_size))

    def on_epoch_end(self):
        np.random.shuffle(self.dish_ids)

    def __getitem__(self, index):
        batch_dish_ids = self.dish_ids[index * self.batch_size:(index + 1) * self.batch_size]

        rgb_imgs, depth_raw_imgs, depth_color_imgs, frames = [], [], [], []
        mass_labels, macro_labels = [], []

        for dish_id, frame_image in batch_dish_ids:
            dish_row = self.dish_data[self.dish_data['dish_id'] == dish_id].iloc[0]
            mass_labels.append(dish_row['total_mass'])
            # Make sure macro_labels is 2D with shape [batch_size, 4]
            macro_labels.append([
                dish_row['total_calories'],
                dish_row['total_fat'],
                dish_row['total_carb'],
                dish_row['total_protein']
            ])

            # Load overhead images
            rgb_imgs.append(self.preprocess_image(os.path.join(OVERHEAD_DIR, f"{dish_id}", "rgb.png")))
            depth_raw_imgs.append(self.preprocess_image(os.path.join(OVERHEAD_DIR, f"{dish_id}", "depth_raw.png")))
            depth_color_imgs.append(self.preprocess_image(os.path.join(OVERHEAD_DIR, f"{dish_id}", "depth_color.png")))

            # Load frames (only 1 frame used)
            frame_path = os.path.join(FRAMES_DIR, f"{dish_id}", "frames_sampled30", frame_image)
            frames.append(self.preprocess_image(frame_path))

        return (
            {
                "rgb_input": np.array(rgb_imgs),
                "depth_raw_input": np.array(depth_raw_imgs),
                "depth_color_input": np.array(depth_color_imgs),
                "frame_input": np.array(frames),
            },
            {
                "mass_output": np.array(mass_labels),
                "macro_output": np.array(macro_labels),
            }
        )

    def preprocess_image(self, image_path):
        img = image.load_img(image_path, target_size=IMAGE_SIZE)
        img = image.img_to_array(img) / 255.0
        return img

# Step 4: Prepare the training, validation, and test generators
train_ids = train_dishes
val_ids = val_dishes
test_ids = test_dishes

train_gen = MultiInputGenerator(dish_data, BATCH_SIZE, train_ids)
val_gen = MultiInputGenerator(dish_data, BATCH_SIZE, val_ids)
test_gen = MultiInputGenerator(dish_data, BATCH_SIZE, test_ids)

In [ ]:
# Model definition (Updated)
input_rgb = Input(shape=(224, 224, 3), name="rgb_input")
input_depth_raw = Input(shape=(224, 224, 3), name="depth_raw_input")
input_depth_color = Input(shape=(224, 224, 3), name="depth_color_input")
input_frame = Input(shape=(224, 224, 3), name="frame_input")

# Feature extraction layers (RGB input for weight prediction)
x_rgb = Conv2D(32, (3, 3), activation='relu', padding='same')(input_rgb)
x_rgb = MaxPooling2D((2, 2))(x_rgb)
x_rgb = Conv2D(64, (3, 3), activation='relu', padding='same')(x_rgb)
x_rgb = MaxPooling2D((2, 2))(x_rgb)
x_rgb = Conv2D(128, (3, 3), activation='relu', padding='same')(x_rgb)
x_rgb = MaxPooling2D((2, 2))(x_rgb)
x_rgb = GlobalAveragePooling2D()(x_rgb)

# Feature extraction layers (Depth Raw input for weight prediction)
x_depth_raw = Conv2D(32, (3, 3), activation='relu', padding='same')(input_depth_raw)
x_depth_raw = MaxPooling2D((2, 2))(x_depth_raw)
x_depth_raw = Conv2D(64, (3, 3), activation='relu', padding='same')(x_depth_raw)
x_depth_raw = MaxPooling2D((2, 2))(x_depth_raw)
x_depth_raw = Conv2D(128, (3, 3), activation='relu', padding='same')(x_depth_raw)
x_depth_raw = MaxPooling2D((2, 2))(x_depth_raw)
x_depth_raw = GlobalAveragePooling2D()(x_depth_raw)

# Feature extraction layers (Depth Color input for weight prediction)
x_depth_color = Conv2D(32, (3, 3), activation='relu', padding='same')(input_depth_color)
x_depth_color = MaxPooling2D((2, 2))(x_depth_color)
x_depth_color = Conv2D(64, (3, 3), activation='relu', padding='same')(x_depth_color)
x_depth_color = MaxPooling2D((2, 2))(x_depth_color)
x_depth_color = Conv2D(128, (3, 3), activation='relu', padding='same')(x_depth_color)
x_depth_color = MaxPooling2D((2, 2))(x_depth_color)
x_depth_color = GlobalAveragePooling2D()(x_depth_color)

# Feature extraction layers (Frame input for macronutrient prediction)
x_frame = Conv2D(32, (3, 3), activation='relu', padding='same')(input_frame)
x_frame = MaxPooling2D((2, 2))(x_frame)
x_frame = Conv2D(64, (3, 3), activation='relu', padding='same')(x_frame)
x_frame = MaxPooling2D((2, 2))(x_frame)
x_frame = Conv2D(128, (3, 3), activation='relu', padding='same')(x_frame)
x_frame = MaxPooling2D((2, 2))(x_frame)
x_frame = GlobalAveragePooling2D()(x_frame)

# Concatenate all feature maps
x = Concatenate()([x_rgb, x_depth_raw, x_depth_color, x_frame])

# Fully connected layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layers
mass_output = Dense(1, name="mass_output")(x)  # Total mass output
macro_output = Dense(4, name="macro_output")(x)  # Four macronutrient outputs: calories, fat, carbs, protein

# Build and compile the model
model = Model(inputs=[input_rgb, input_depth_raw, input_depth_color, input_frame],
              outputs=[mass_output, macro_output])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss={'mass_output': 'mean_squared_error', 'macro_output': 'mean_squared_error'},
              metrics={'mass_output': 'mae', 'macro_output': 'mae'})

In [24]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ rgb_input           │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depth_raw_input     │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depth_color_input   │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ frame_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_151 (Conv2D) │ (None, 224, 224,  │        896 │ rgb_input[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_154 (Conv2D) │ (None, 224, 224,  │        896 │ depth_raw_input[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_157 (Conv2D) │ (None, 224, 224,  │        896 │ depth_color_inpu… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_160 (Conv2D) │ (None, 224, 224,  │        896 │ frame_input[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_55    │ (None, 112, 112,  │          0 │ conv2d_151[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_58    │ (None, 112, 112,  │          0 │ conv2d_154[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_61    │ (None, 112, 112,  │          0 │ conv2d_157[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_64    │ (None, 112, 112,  │          0 │ conv2d_160[0][0]  │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_152 (Conv2D) │ (None, 112, 112,  │     18,496 │ max_pooling2d_55… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_155 (Conv2D) │ (None, 112, 112,  │     18,496 │ max_pooling2d_58… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_158 (Conv2D) │ (None, 112, 112,  │     18,496 │ max_pooling2d_61… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_161 (Conv2D) │ (None, 112, 112,  │     18,496 │ max_pooling2d_64… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_56    │ (None, 56, 56,    │          0 │ conv2d_152[0][0]

 Total params: 768,261 (2.93 MB)

 Trainable params: 768,261 (2.93 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
# Step 5: Train the model
model.fit(train_gen,
          validation_data=val_gen,
          epochs=EPOCHS,
          verbose=1)

# Model evaluation on test set
model.evaluate(test_gen)

c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['rgb_input', 'depth_raw_input', 'depth_color_input', 'frame_input']. Received: the structure of inputs={'rgb_input': '*', 'depth_raw_input': '*', 'depth_color_input': '*', 'frame_input': '*'}
  warnings.warn(


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/mean_squared_error_1/sub/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\CapstoneGithub\CapstoneGithubVenv\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\Rafi Kyandra\AppData\Local\Programs\Python\Py